In [ ]:
# This cell is for WIPs
# Imports
import numpy as np
import pandas as pd
from db import DB
from db import list_profiles
from datetime import *
import os
import glob
import time
from os import listdir
from os.path import isfile, join
from openpyxl import Workbook
from openpyxl import load_workbook
import xlsxwriter

import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

# Pull file names from folder and load as a dataframe
base_path = r'C:\Users\brandon.terrebonne\Desktop\outbound_file_screening' #Will need to change if others use this code
dir_files = [f for f in listdir(base_path) if isfile(join(base_path, f))]
dir_files_final = [s for s in dir_files if s[0] != '~'] # Makes sure we don't pull an excel litter file
df1_data = base_path + '\\' + dir_files_final[0]
raw_invoice_file = pd.read_csv(df1_data)
invoice_file = pd.DataFrame(raw_invoice_file) # Base DF

# Renaming fields if there are name transforms setup for the buyer
invoice_file_fields = invoice_file.columns.values.tolist()

eslap_invoice_fields = ['company_id', 
                 'division_id',
                 'invoice_id',
                 'amount',
                 'currency',
                 'payment_due_date',
                 'transaction_type',
                 'transaction_date',
                 'voucher_id',
                 'payment_term',
                 'payment_method',
                 'adj_invoice_id',
                 'adjustment_reason_code',
                 'vat_amount']

def buyer_outbound_mappings(buyer_configs_data, buyer_name, table_field_name, eslap_col_names):
    buyer_data = buyer_configs_data[buyer_configs_data['upload_directory'] == buyer_name]
    index_list = buyer_data.index.tolist()
    buyer_index = index_list[0]
    pre_transform_col_names = []
    pre_transform_col_names_values = []
    for x in eslap_col_names:
        try:
            trans_name = buyer_data.loc[buyer_index][table_field_name]['TransformItems'][x]['IncomingColumnName']
            pre_transform_col_names.append(trans_name)
            pre_transform_col_names_values.append(x)
        except:
            pass
    file_mapping_dict = dict(zip(pre_transform_col_names, pre_transform_col_names_values))
    return file_mapping_dict

db = DB(profile='uat')
buyer_configs = db.query("""
                        select *
                        from fileset_configuration
                        ;"""
                        )

print sorted(buyer_configs['upload_directory'].tolist())
buyer_name_input = raw_input('\nBuyer Name:').lower()
invoice_col_names_transformed = buyer_outbound_mappings(buyer_configs, buyer_name_input, 'invoice_map', eslap_invoice_fields) # Will need updating onces the scope of this code is extended to org/user files

final_col_name_list = []
for x in invoice_file_fields:
    if x in invoice_col_names_transformed:
        final_col_name_list.append(invoice_col_names_transformed[x])
    else:
        final_col_name_list.append(x)
        
invoice_file.rename(columns=dict(zip(invoice_file_fields, final_col_name_list)), inplace=True)

# Formatting invoice_file columns
invoice_file['amount'] = invoice_file['amount'].astype(str)
invoice_file['adj_invoice_id'] = invoice_file['adj_invoice_id'].astype(str)
invoice_file['invoice_id'] = invoice_file['invoice_id'].astype(str)

# Creating two DFs, one for each transaction_type
trans_type1_df = invoice_file[invoice_file['transaction_type'] == 1]
trans_type2_df = invoice_file[invoice_file['transaction_type'] == 2]
invoice_file_rowcount = len(invoice_file.index)
trans_type1_df_rowcount = len(trans_type1_df.index)
trans_type2_df_rowcount = len(trans_type2_df.index)

# Counts on matched vs unmatched adjustments
unmatched_adj_df = trans_type2_df[trans_type2_df['adj_invoice_id'] == 'nan']
matched_adj_df = trans_type2_df[trans_type2_df['adj_invoice_id'] != 'nan']
matched_inv_df = trans_type1_df[trans_type1_df['adj_invoice_id'] != 'nan']

unmatched_count = len(unmatched_adj_df.index)
matched_count = len(matched_adj_df.index)
matched_inv_count = len(matched_inv_df.index)

# Counts on future-dated invoices and adjustments
todays_date = datetime.today().strftime('%Y-%m-%d')

trans_type1_df['payment_due_date'] = trans_type1_df['payment_due_date'].astype(datetime)
future_dated_type1 = trans_type1_df[trans_type1_df['payment_due_date'] > todays_date]
future_dated_type1_count = len(future_dated_type1.index) # Count of future-dated invoices

matched_adj_df['payment_due_date'] = matched_adj_df['payment_due_date'].astype(datetime)
future_dated_matched_df = matched_adj_df[matched_adj_df['payment_due_date'] > todays_date]
future_dated_matched_count = len(future_dated_matched_df.index) # count of future-dated matched adjustments

unmatched_adj_df['payment_due_date'] = unmatched_adj_df['payment_due_date'].astype(datetime)
future_dated_unmatched_df = unmatched_adj_df[unmatched_adj_df['payment_due_date'] > todays_date]
future_dated_unmatched_count = len(future_dated_unmatched_df.index) # count of future-dated unmatched adjustments

# Print record count summary
print "---Invoice File Summary---\nFile name: %s" % (dir_files_final[0])
print "Number of records in the file: %s" % (invoice_file_rowcount)
print "Number of invoices: %s" % (trans_type1_df_rowcount)
print "Number of adjustments: %s" % (trans_type2_df_rowcount)
print "Number of unmatched adjustments: %s" % (unmatched_count)
print "Number of matched adjusments: %s\n" % (matched_count)

# Print age summary
print "---Age Summary for %s---\nNumber of future dated invoices: %s" % (todays_date, future_dated_type1_count)
print "Number of future dated matched adjustments: %s" % (future_dated_matched_count)
print "Number of future dated unmatched adjustments:%s\n\n---Invoice File Issue Summary---" % (future_dated_unmatched_count)
    
# cast 'amount' as float for each sub-DF
trans_type1_df['amount'] = trans_type1_df['amount'].astype(float)
trans_type2_df['amount'] = trans_type2_df['amount'].astype(float)

# Matched adjustment counts
trans_type2_df['adj_invoice_id'] = trans_type2_df['adj_invoice_id'].astype(str)
unmatched_adj_df = trans_type2_df[trans_type2_df['adj_invoice_id'] == 'nan']
matched_adj_df = trans_type2_df[trans_type2_df['adj_invoice_id'] != 'nan']

unmatched_count = len(unmatched_adj_df.index)
matched_count = len(matched_adj_df.index)

inv_id_list = invoice_file['invoice_id'].tolist()
adj_inv_id_list = matched_adj_df['adj_invoice_id'].tolist()
valid_matched_count = len([i for i in adj_inv_id_list if i in inv_id_list])
invalid_matched = [i for i in adj_inv_id_list if i not in inv_id_list]
invalid_matched_df = invoice_file[invoice_file['adj_invoice_id'].isin(invalid_matched)]

# Number of records with 'amount' equal to zero
invoice_file['amount'] = invoice_file['amount'].astype(float)
zero_amounts = invoice_file[invoice_file['amount'] == 0]
zero_amounts_count = len(zero_amounts.index)

# Number of records incorrectly applying transaction_type logic
negative_trans_type1s = trans_type1_df[trans_type1_df['amount'] < 0]
positive_trans_type2s = trans_type2_df[trans_type2_df['amount'] > 0]

# Print warnings
if len(invalid_matched_df.index) > 0:
    print "Warning - Number of matched adjustments that do not have their corresponding invoices present: %s" % (len(invalid_matched_df.index))
else:
    print "Passed - All matched adjustments have their corresponding invoices present"
if zero_amounts_count > 0:
    print "Warning - Number of records with 'amount' equal to zero: %s" % (zero_amounts_count)
else:
    print "Passed - No records with amount = 0.00"
if len(negative_trans_type1s.index) > 0:
    print "Warning - Number of NEGATIVE transaction_type 1s: %s" % (len(negative_trans_type1s.index))
else:
    print "Passed - All transaction_type 1s are positive"
if len(positive_trans_type2s.index) > 0:
    print "Warning - Number of POSITIVE transaction_type 2s: %s" % (len(positive_trans_type2s.index))
else:
    print "Passed - All transaction_type 2s are negative"
if matched_inv_count > 0:
    print "Warning - Number of transaction_type 1s with adj_invoice_ids: %s" % (matched_inv_count)
else:
    print "Passed - No invoices found with adj_invoice_id field populated"
    
# List all fields that are populated < 1 time
inv_file_fields = list(invoice_file.columns.values) #get list of field names
empty_fields = []
inv_file_len = len(invoice_file.index) #get row count of invoice file

for x in inv_file_fields:
    invoice_file[x] = invoice_file[x].astype(str)
    new_df = invoice_file[invoice_file[x] == 'nan']
    if len(new_df.index) == inv_file_len:
        empty_fields.append(x)

if not empty_fields:
    print "All fields are populated at least once."
if empty_fields:
    print "Warning - Fields not populated: %s" % (empty_fields)
    
# Print currency pivot table
pd.set_option('display.float_format', lambda x: '%.2f' % x)

invoice_file['amount'] = invoice_file['amount'].astype(float)
currency_pivot = invoice_file.pivot_table(index=['currency'], values=['amount'], aggfunc = np.sum)
print "\n---Currency Breakdown---\n%s" % (currency_pivot)

# Excel for costcouk =CONCAT(LEFT(E2,4),"-",MID(E2,5,2),"-",RIGHT(E2,2))

# Writing to an xlsx
summary_file_path = base_path + '\\' + 'SUMMARY_' + dir_files_final[0] + '.xlsx'
workbook = xlsxwriter.Workbook(summary_file_path)
# worksheet = workbook.add_worksheet()

workbook.close()

wb = load_workbook(summary_file_path)
ws1 = wb.create_sheet('Summary', 0) # insert at first position

for i in range(1,101):
    for j in range(1,101):
        ws1.cell(row=i, column=j)

workbook = xlsxwriter.Workbook(summary_file_path)
# worksheet = workbook.add_worksheet()
workbook.close()

ws1['A1'] = '--- Invoice File Summary ---'
ws1['A2'] = 'File name: %s' % (dir_files_final[0])
ws1['A3'] = 'Number of records in the file: %s' % (invoice_file_rowcount)
ws1['A4'] = 'Number of invoices: %s' % (trans_type1_df_rowcount)
ws1['A5'] = 'Number of adjustments: %s' % (trans_type2_df_rowcount)
ws1['A6'] = 'Number of unmatched adjustments: %s' % (unmatched_count)
ws1['A7'] = 'Number of matched adjusments: %s' % (matched_count)

ws1['A9'] = '--- Age Summary for %s ---' % (todays_date)
ws1['A10'] = 'Number of future dated invoices: %s' % (future_dated_type1_count)
ws1['A11'] = 'Number of future dated unmatched adjustments: %s' % (future_dated_unmatched_count)
ws1['A12'] = 'Number of future dated matched adjustments: %s' % (future_dated_matched_count)

ws1['A14'] = '--- Issue Summary ---'
if len(invalid_matched_df.index) > 0:
    ws1['A15'] = 'WARNING - Number of matched adjustments missing their corresponding invoices: %s' % (len(invalid_matched_df.index))
else:
    ws1['A15'] = 'Passed - Number of matched adjustments missing their corresponding invoices: %s' % (len(invalid_matched_df.index))
if zero_amounts_count > 0:
    ws1['A16'] = 'WARNING - Number of records with \'amount\' equal to zero: %s' % (zero_amounts_count)
else:
    ws1['A16'] = 'Passed - Number of records with \'amount\' equal to 0.00: %s' % (zero_amounts_count)
if len(negative_trans_type1s.index) > 0:
    ws1['A17'] = 'WARNING - Number of NEGATIVE transaction_type 1s: %s' % (len(negative_trans_type1s.index))
else: 
    ws1['A17'] = 'Passed - Number of NEGATIVE transaction_type 1s: %s' % (len(negative_trans_type1s.index))
if len(positive_trans_type2s.index) > 0:
    ws1['A18'] = 'WARNING - Number of POSITIVE transaction_type 2s: %s' % (len(positive_trans_type2s.index))
else:
    ws1['A18'] = 'Passed - Number of POSITIVE transaction_type 2s: %s' % (len(positive_trans_type2s.index))
if matched_inv_count > 0:
    ws1['A19'] = "Warning - Number of transaction_type 1s with adj_invoice_ids: %s" % (matched_inv_count)
else:
    ws1['A19'] = "Passed - No invoices found with adj_invoice_id field populated"
if not empty_fields:
    ws1['A20'] = 'All fields are populated at least once.'
if empty_fields:
    ws1['A20'] = 'Warning - Fields not populated: %s' % (empty_fields)

wb.save(summary_file_path)

#DFs to excel worksheets... http://stackoverflow.com/questions/20219254/how-to-write-to-an-existing-excel-file-without-overwriting-data-using-pandas
book = load_workbook(summary_file_path)
writer = pd.ExcelWriter(summary_file_path, engine='openpyxl')
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

inv_file_tab_name = str(dir_files_final[0])[:30]

raw_invoice_file.to_excel(writer, sheet_name= inv_file_tab_name, index=False)
if future_dated_type1_count > 0:
    future_dated_type1.to_excel(writer, sheet_name='Future_Dated_Inv', index= False)
if unmatched_count > 0:
    unmatched_adj_df.to_excel(writer, sheet_name='Unmatched_Adj', index= False)
if matched_count > 0:
    matched_adj_df.to_excel(writer, sheet_name='Matched_Adj', index= False)
if future_dated_unmatched_count > 0:
    future_dated_unmatched_df.to_excel(writer, sheet_name='Future_Dated_Unmatched', index= False)
if future_dated_matched_count > 0:
    future_dated_matched_df.to_excel(writer, sheet_name='Future_Dated_Matched', index= False)
if matched_inv_count > 0:
    matched_inv_df.to_excel(writer, sheet_name='Matched_Invoices', index= False)
if len(invalid_matched_df.index) > 0:
    invalid_matched_df.to_excel(writer, sheet_name='Bad_Matched_Adjs', index= False)


sh= book.get_sheet_by_name('Sheet1')
book.remove_sheet(sh)

writer.save()